# An introduction to remote calls in Julia

First add some workers

In [ ]:
# Use this to add workers locally
# if nprocs() == 1 
#     addprocs(2)
# end;

# Use this to add workers on JuliaBox
using JuliaRunClient
initializeCluster(2)
# initializeCluster(16) # more workers can be added with a larger subscription

workers()

Now we create an anonymous function `() -> "Allez les Bleus"` and ask Julia to execute it on worker `2` with `remoteall`.

In [ ]:
r = remotecall(() -> "Allez les Bleus", 2)

The result return from `remotecall` is a `Future`. The `Future` keeps a reference to the the actual result of the computation. To result can be retrieved with the `fetch` function.

In [ ]:
fetch(r)

but this doesn't have to happen on the master node

In [ ]:
remotecall(() -> println(fetch(r)), 2);

In some cases, this could also happen from another worker process but JuliaBox is by default set up to only allow master-worker communication so

In [ ]:
remotecall(() -> println(fetch(r)), 3);

will fail on JuliaBox but not if you e.g. `addprocs(2)` locally.

In [ ]:
r = remotecall(() -> myid(), 3)
@show fetch(r)
remotecall(() -> println(fetch(r)), 2);

In [ ]:
r = remotecall(i -> i + myid(), 2, 1)
fetch(r)

## Tasks

The `remotecall` is asynchronous and synchronization will happen when the result is fetched. Often, it is useful to be able to control the synchronization more precisely. This is possible with the two related functions `remotecall_fetch` and `remotecall_wait`. Their behaviors are very similar to `remotecall` but both are blocking. However, they may be combined with `@async` and `@sync` annotations to setup blocks of code to be run asynchronously and when sync up again. The difference between the two blocking versions is that `remotecall_fetch` will return the result whereas `remotecall_wait` will return a `Future` with a reference to the result.

The blocking behavior can be tested with the `sleep` function

In [ ]:
@time remotecall(() -> sleep(2), 2);

In [ ]:
@time fetch(remotecall(() -> sleep(2), 2));

In [ ]:
@time typeof(remotecall_wait(() -> sleep(2), 2))

In [ ]:
@time typeof(remotecall_fetch(() -> sleep(2), 2))

By annotating remotecall_fetch with `@async`, we can make the code block return immediately

In [ ]:
@time @async remotecall_fetch(() -> (sleep(2); 1), 2)

This can be useful when we want to run concurrent computations on the available workers. First we show an example without asynchronous execution and the one with asynchronous execution

In [ ]:
@time for p in workers()
    remotecall_fetch(() -> sleep(1), p)
end

In [ ]:
@time @sync for p in workers()
    @async remotecall_fetch(() -> sleep(1), p)
end

This is a very common pattern so Julia has a convenience function defined for exactly this

In [ ]:
@time @sync asyncmap(p -> remotecall_fetch(() -> (sleep(1); myid()), p), workers())

The nested anonymous functions become hard to read. This is true in particular when the function body of the anonymous functions become non-trivial. It is therefore often better to use Julia's `do` syntax for anonymous functions. An equivalent formulation of the previous call with `do` syntax would look like

In [ ]:
@time @sync asyncmap(workers()) do p
    remotecall_fetch(p) do
        return (sleep(1); myid())
    end
end

It might take a little effort to get used to the `do` syntax but doing so

In [ ]:
[remotecall_fetch(() -> myid(), i) for i in workers()]

In [ ]:
remotecall_wait(3) do
    println("do syntax")
end